# TensorBoard
In this tutorial, we are going to implement a neural network with fully-connected layers to perform classification, visualize the model and plot the loss and gradients by using a tensorboard.

![alt text](jpg/tensorboard.jpg "model image")

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


mnist = input_data.read_data_sets("./mnist", one_hot=True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 784)
y_train:  (55000, 10)
x_test:  (10000, 784)
y_test:  (10000, 10)


## Fully-connected layer
To implement a fully-connected layer, we can simply use `tf.matmul` function for 2D matrix multiplication. In this code, we are using `tf.variable_scope` and `tf.get_variable` functions to encapsulate and manage tensor variables effectively. Also, We are choosing a `tf.random_uniform_initializer` as variable initializer for our model. There are some other variable initializers such as `tf.random_normal_initializer and tf.truncated_normal_initializer`. 

In [2]:
def fully_connected(x, dim_in, dim_out, name):
    with tf.variable_scope(name) as scope:
        # create variables
        w = tf.get_variable('w', shape=[dim_in, dim_out], 
                            initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        b = tf.get_variable('b', shape=[dim_out])
        
        # create operations
        out = tf.matmul(x, w) + b
        
        return out    

## Neural network
Now, we will develope a neural network with 2 hidden layers using a `fully_connected` function. In this code, we are using a `tf.nn.relu` as our non-linear activation function.

In [3]:
# Create model
def neural_network(x, dim_in=784, dim_h=500, dim_out=10):
    # 1st hidden layer with ReLU
    h1 = fully_connected(x, dim_in, dim_h, name='hidden_layer_1')
    h1 = tf.nn.relu(h1)
    
    # 2nd hidden layer with ReLU
    h2 = fully_connected(h1, dim_h, dim_h, name='hidden_layer_2')
    h2 = tf.nn.relu(h2)
    
    # output layer with linear
    out = fully_connected(h2, dim_h, dim_out, name='output_layer')
    
    return out

## Place holder
To train the neural network with mini-batch gradient descent, placeholders should be defined for mini-batch input data and target data. In addition, None type is used so that any batch size of data can be fed into the placeholder.

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

## Construct graph
Now, we will build a graph for our neural network. Note that you should run the cell below only once. If you run this more than once, an error message will be printed out: `"ValueError: Variable h1/w already exists, disallowed."`. This is because we used `tf.get_variable` above and this function doesn't allow creating variables with the existing names. 
To fix this problem, you can just type `tf.get_variable_scope().reuse_variables()` before cell below and run it. Then, `tf.get_variable`  will use previously created tensor variables instead of trying to create new ones.

In [5]:
# Construct model with default value
out = neural_network(x)

## Loss, Optimizer and Summary
To train the neural network, we should implement loss (this is a tensor) and optimizer (this is an operator). We can use `tf.nn.softmax_cross_entropy_with_logits` function to compute the loss. This function expects unscaled logits, since it performs a softmax on logits internally for efficiency. Do not call this op with the output of softmax, as it will produce incorrect results.
Also, we choose `tf.train.RMSPropOptimizer` as our optimizer to minimize the loss.

In [6]:
# loss 
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))

# accuracy
with tf.name_scope('accuracy'):
    pred = tf.argmax(out, 1)
    target = tf.argmax(y, 1)
    correct_pred = tf.equal(pred, target)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    
# train op
with tf.name_scope('optimizer'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
    grads = tf.gradients(loss, tf.trainable_variables())
    grads_and_vars = list(zip(grads, tf.trainable_variables()))
    train_op = optimizer.apply_gradients(grads_and_vars=grads_and_vars)

# add summary op   
tf.scalar_summary('batch_loss', loss)
for var in tf.trainable_variables():
    tf.histogram_summary(var.op.name, var)
for grad, var in grads_and_vars:
    tf.histogram_summary(var.op.name+'/gradient', grad)

summary_op = tf.merge_all_summaries() 

## Session: train and test model
From above, we build our neural network model to classify the MNIST dataset. To launch our model, we will implement a session where the model is actually trained and tested on the MNIST dataset.

** Train **

First, we initialize all variables we created above. 
This can be done by running [tf.initialize_all_variables()](https://www.tensorflow.org/versions/r0.11/api_docs/python/state_ops.html#initialize_all_variables). All variables in the default graph are initialized with `tf.random_uniform_initializer` in our case.
The most important part of code in the training phase is `sess.run([train_op, loss], feed_dict={x: x_batch, y:y_batch})`. This part of code feeds mini-batch data into placeholder and run optimizer to update variables with `tf.train.RMSPropOptimizer` once. 
Also, loss is evaluated to print out the average loss for each epoch.

** Test **

Testing phase is quite simple. We use `sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels})` to print out the test accuracy. 

In [7]:
batch_size = 100
log_path = 'log/'

# launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # initialize tensor variables
    tf.initialize_all_variables().run()
    summary_writer = tf.train.SummaryWriter(log_path, graph=tf.get_default_graph())
    # training cycle
    for epoch in range(15):
        avg_loss = 0.
        n_iters_per_epoch = int(mnist.train.num_examples / batch_size)
        # loop over all batches
        for i in range(n_iters_per_epoch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            # run optimization op (backprop) and loss op (to get loss value)
            feed_dict={x: x_batch, y: y_batch}
            _, c = sess.run([train_op, loss], feed_dict=feed_dict)
            # compute average loss
            avg_loss += c / n_iters_per_epoch
            
            if i % 10 == 0:
                summary = sess.run(summary_op, feed_dict)
                summary_writer.add_summary(summary, epoch*n_iters_per_epoch + i)
        print "Epoch %d, Loss: %.3f"% (epoch+1, avg_loss)
    print "Finished training!"
    
    print ""
    print "Test accuracy:", sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels})

Epoch 1, Loss: 0.573
Epoch 2, Loss: 0.108
Epoch 3, Loss: 0.065
Epoch 4, Loss: 0.044
Epoch 5, Loss: 0.032
Epoch 6, Loss: 0.023
Epoch 7, Loss: 0.019
Epoch 8, Loss: 0.015
Epoch 9, Loss: 0.012
Epoch 10, Loss: 0.010
Epoch 11, Loss: 0.008
Epoch 12, Loss: 0.008
Epoch 13, Loss: 0.007
Epoch 14, Loss: 0.006
Epoch 15, Loss: 0.005
Finished training!

Test accuracy: 0.981


<br>
## Execute the TensorBoard
To execute the tensorboard, open the new terminal, run command below and open http://localhost:6005/ into your web browser.
```bash
$ tensorboard --logdir='./log' --port=6005 
```